In [5]:
import numpy as np
import pickle
from pathlib import Path
import pandas as pd

In [6]:
# Config: Forecast 001 (prices)
forecast_file = Path("forecasts/forecast_001.pkl")
dgp_name = "gbm_low_vol"
dgp_type = "prices"

print(f"\n[1] Loading Forecast → {forecast_file}")
with open(forecast_file, "rb") as f:
    forecast_result = pickle.load(f)

print("\n[Forecast Contents]")
if isinstance(forecast_result, (list, tuple)) and len(forecast_result) == 5:
    low, median, high, samples, base_price = forecast_result
    print(f"  Low shape:     {low.shape}")
    print(f"  Median shape:  {median.shape}")
    print(f"  High shape:    {high.shape}")
    print(f"  Samples shape: {samples.shape}")
    print(f"  Base price:    {base_price} ({type(base_price).__name__})")
else:
    print("❌ Unexpected forecast format:")
    print(forecast_result)

# Load DGP paths
price_path_file = Path(f"datasets/{dgp_name}_paths.npy")
return_path_file = Path(f"datasets/{dgp_name}_returns_paths.npy")

print("\n[DGP Paths]")
if price_path_file.exists():
    price_paths = np.load(price_path_file)
    print(f"  ✅ Price paths loaded:  {price_path_file.name} → {price_paths.shape}")
else:
    print(f"  ❌ Price paths not found: {price_path_file}")

if return_path_file.exists():
    return_paths = np.load(return_path_file)
    print(f"  ✅ Return paths loaded: {return_path_file.name} → {return_paths.shape}")
else:
    print(f"  ❌ Return paths not found: {return_path_file}")


[1] Loading Forecast → forecasts/forecast_001.pkl

[Forecast Contents]
  Low shape:     (22,)
  Median shape:  (22,)
  High shape:    (22,)
  Samples shape: (1000, 22)
  Base price:    108.7803232 (float64)

[DGP Paths]
  ✅ Price paths loaded:  gbm_low_vol_paths.npy → (1000, 22)
  ✅ Return paths loaded: gbm_low_vol_returns_paths.npy → (1000, 22)


In [7]:
# Config: Forecast 002 (returns)
forecast_file = Path("forecasts/forecast_002.pkl")
dgp_type = "returns"

print(f"\n[2] Loading Forecast → {forecast_file}")
with open(forecast_file, "rb") as f:
    forecast_result = pickle.load(f)

print("\n[Forecast Contents]")
if isinstance(forecast_result, (list, tuple)) and len(forecast_result) == 5:
    low, median, high, samples, base_value = forecast_result
    print(f"  Low shape:     {low.shape}")
    print(f"  Median shape:  {median.shape}")
    print(f"  High shape:    {high.shape}")
    print(f"  Samples shape: {samples.shape}")
    print(f"  Base value:    {base_value} (ignored)")
else:
    print("❌ Unexpected forecast format:")
    print(forecast_result)

# Load DGP returns
return_path_file = Path(f"datasets/{dgp_name}_returns_paths.npy")
print("\n[DGP Return Paths]")

if return_path_file.exists():
    return_paths = np.load(return_path_file)
    print(f"  ✅ Loaded: {return_path_file.name} → {return_paths.shape}")
    print("\n[Shape Check]")
    if return_paths.shape == samples.shape:
        print("  ✅ Forecast and DGP return shapes match!")
    else:
        print("  ❌ Shape mismatch!")
        print(f"     Forecast shape: {samples.shape}")
        print(f"     DGP shape:      {return_paths.shape}")
else:
    print(f"  ❌ Return paths not found: {return_path_file}")


[2] Loading Forecast → forecasts/forecast_002.pkl

[Forecast Contents]
  Low shape:     (22,)
  Median shape:  (22,)
  High shape:    (22,)
  Samples shape: (1000, 22)
  Base value:    0.00285448 (ignored)

[DGP Return Paths]
  ✅ Loaded: gbm_low_vol_returns_paths.npy → (1000, 22)

[Shape Check]
  ✅ Forecast and DGP return shapes match!


In [8]:
# Price Start Diagnostic
prices_csv_path = Path(f"datasets/{dgp_name}_prices.csv")
paths_npy_path = Path(f"datasets/{dgp_name}_paths.npy")

print("\n[3] Price Path Diagnostic")
if not prices_csv_path.exists() or not paths_npy_path.exists():
    print("❌ Required files not found.")
else:
    historical_prices = pd.read_csv(prices_csv_path).squeeze()
    last_price = historical_prices.iloc[-1]
    simulated_paths = np.load(paths_npy_path)

    print(f"  Last historical price: {last_price:.8f}")
    print(f"  Simulated paths shape: {simulated_paths.shape}")

    first_column = simulated_paths[:, 0]
    match = np.allclose(first_column, last_price, rtol=0, atol=1e-8)

    print(f"\n  ➤ First column equals last price? {match}")
    if not match:
        unique_start_prices = np.unique(first_column)
        mean_deviation = np.mean(np.abs(first_column - last_price))
        print(f"  ⚠️ Unique start prices (first 5): {unique_start_prices[:5]}")
        print(f"  ⚠️ Mean deviation from last price: {mean_deviation:.10f}")


[3] Price Path Diagnostic
  Last historical price: 108.78032320
  Simulated paths shape: (1000, 22)

  ➤ First column equals last price? False
  ⚠️ Unique start prices (first 5): [107.14900163 107.18790803 107.19726354 107.24064519 107.24310287]
  ⚠️ Mean deviation from last price: 0.4418434255


In [9]:
# Return Start Diagnostic
returns_csv_path = Path(f"datasets/{dgp_name}_returns.csv")
returns_paths_npy = Path(f"datasets/{dgp_name}_returns_paths.npy")

print("\n[4] Return Path Diagnostic")
if not returns_csv_path.exists() or not returns_paths_npy.exists():
    print("❌ Required files not found.")
else:
    historical_returns = pd.read_csv(returns_csv_path).squeeze()
    last_return = historical_returns.iloc[-1]
    simulated_returns = np.load(returns_paths_npy)

    print(f"  Last historical return: {last_return:.8f}")
    print(f"  Simulated return paths shape: {simulated_returns.shape}")

    first_column = simulated_returns[:, 0]
    match = np.allclose(first_column, last_return, rtol=0, atol=1e-8)

    print(f"\n  ➤ First column equals last return? {match}")
    if not match:
        unique_start_returns = np.unique(first_column)
        mean_deviation = np.mean(np.abs(first_column - last_return))
        print(f"  ⚠️ Unique start returns (first 5): {unique_start_returns[:5]}")
        print(f"  ⚠️ Mean deviation from last return: {mean_deviation:.10f}")


[4] Return Path Diagnostic
  Last historical return: 0.00285448
  Simulated return paths shape: (1000, 22)

  ➤ First column equals last return? False
  ⚠️ Unique start returns (first 5): [-0.01511006 -0.01474702 -0.01465974 -0.01425514 -0.01423222]
  ⚠️ Mean deviation from last return: 0.0047165397
